Simple CNN example for MNIST dataset. Adapted from:
https://github.com/pytorch/examples/tree/main/mnist

The original pytorch code is BSD-3 licensed:
https://github.com/pytorch/examples/blob/main/LICENSE

In [1]:
# import dependencies

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

In [9]:
# set up model

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 2)
        self.fc2 = nn.Linear(2, 2)
        self.fc3 = nn.Linear(2, 10)

    def forward(self, x):
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        output = F.log_softmax(x, dim=1)
        return output

In [4]:
# training and testing functions

def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


In [5]:
# parameters controlling the training run

class args: pass
args.batch_size = 64
args.test_batch_size = 1000
args.epochs = 4
args.lr = 1.0 # learning rate
args.gamma = 0.7 # learning rate step gamma
args.seed = 1234 # random seed
args.log_interval = 10 # how frequently is training status reported
args.no_cuda = False  # set to True to not use CUDA for training (even if available)
args.no_mps = False # set to True to not use Mac OS GPUs for training (even if available)


In [10]:
torch.manual_seed(args.seed)

use_cuda = not args.no_cuda and torch.cuda.is_available()
use_mps = not args.no_mps and torch.backends.mps.is_available()

if use_cuda:
    device = torch.device("cuda")
elif use_mps:
    device = torch.device("mps")
else:
    device = torch.device("cpu")

train_kwargs = {'batch_size': args.batch_size}
test_kwargs = {'batch_size': args.test_batch_size}
if use_cuda:
    cuda_kwargs = {'num_workers': 1,
                   'pin_memory': True,
                   'shuffle': True}
    train_kwargs.update(cuda_kwargs)
    test_kwargs.update(cuda_kwargs)

transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

dataset1 = datasets.MNIST('../data', train=True, download=True, transform=transform)
dataset2 = datasets.MNIST('../data', train=False, transform=transform)
train_loader = torch.utils.data.DataLoader(dataset1, **train_kwargs)
test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

model = Net().to(device)
optimizer = optim.Adadelta(model.parameters(), lr=args.lr)

scheduler = StepLR(optimizer, step_size=1, gamma=args.gamma)
for epoch in range(1, args.epochs + 1):
    train(args, model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    scheduler.step()


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.404088
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.342793
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.155957
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.106260
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.081458
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.122114
Train Epoch: 1 [3840/60000 (6%)]	Loss: 1.867821
Train Epoch: 1 [4480/60000 (7%)]	Loss: 1.964791
Train Epoch: 1 [5120/60000 (9%)]	Loss: 1.953545
Train Epoch: 1 [5760/60000 (10%)]	Loss: 1.788341
Train Epoch: 1 [6400/60000 (11%)]	Loss: 1.950446
Train Epoch: 1 [7040/60000 (12%)]	Loss: 1.727705
Train Epoch: 1 [7680/60000 (13%)]	Loss: 1.753153
Train Epoch: 1 [8320/60000 (14%)]	Loss: 1.735670
Train Epoch: 1 [8960/60000 (15%)]	Loss: 1.615628
Train Epoch: 1 [9600/60000 (16%)]	Loss: 1.665292
Train Epoch: 1 [10240/60000 (17%)]	Loss: 1.807980
Train Epoch: 1 [10880/60000 (18%)]	Loss: 1.697303
Train Epoch: 1 [11520/60000 (19%)]	Loss: 1.837456
Train Epoch: 1 [12160/60000 (20%)]	Loss: 1.680373
Train Epoch: 1 [12800/60000 (

In [ ]:
# save the trained model
torch.save(model.state_dict(), "mnist_mlp.pt")